In [9]:
import pandas as pd

In [7]:
import requests 

import numpy as np
import scipy.stats as si

import datetime

def BSM_Price(S, K, T, r, sigma, putcall):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if putcall == 'C':
        result = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    elif putcall == 'P':
        result = (K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0) - S * si.norm.cdf(-d1, 0.0, 1.0))
    return result

def BSM_IV(S, K, T, r, price, putcall):
    thisIV = 1
    dIV = 0.01
    thresholdPx = 0.001
    for i in range(20):
        thisPx = BSM_Price(S, K, T, r, thisIV, putcall)
        dPx = BSM_Price(S, K, T, r, thisIV + dIV, putcall) - thisPx
        thisIV = thisIV + (price - thisPx) / dPx * dIV
        if dPx <= thresholdPx:
            break
    return thisIV

def DateStrToInt(dateStr):
    return (datetime.datetime.strptime(dateStr + 'UTC08', '%d%b%y%Z%H') - datetime.datetime.utcnow()).total_seconds() / 86400

def GetOptionBook():
    urlOptionBook = "https://www.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=option"
    jsonOptionBook = requests.get(url = urlOptionBook).json() 
    resultOptionBook = jsonOptionBook['result']
    for i in resultOptionBook:
        instrument_name = i['instrument_name']
        instrument_data = instrument_name.split("-")
        underlying = instrument_data[0]
        expiry = instrument_data[1]
        strike = float(instrument_data[2])
        putcall = instrument_data[3]
        TTM = DateStrToInt(expiry)/365.25
        underlying_price = float(i['underlying_price'])
        bid_price = i['bid_price']
        ask_price = i['ask_price']
        mid_price = i['mid_price']

        if bid_price==None:
            i['bid_IV'] = None
        else:
            i['bid_IV'] = BSM_IV(underlying_price,strike,TTM,0,float(bid_price)*underlying_price,putcall)

        if ask_price==None:
            i['ask_IV'] = None
        else:
            i['ask_IV'] = BSM_IV(underlying_price,strike,TTM,0,float(ask_price)*underlying_price,putcall)

        if mid_price==None:
            i['mid_IV'] = None
        else:
            i['mid_IV'] = BSM_IV(underlying_price,strike,TTM,0,float(mid_price)*underlying_price,putcall)
        #print(i['mid_IV'] )

    return resultOptionBook
    #print([i['instrument_name'] for i in resultOptionBook])

def GetOptionMeta():
    urlOptionBook = "https://www.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=option"
    jsonOptionBook = requests.get(url = urlOptionBook).json() 
    resultOptionBook = jsonOptionBook['result']
    dictMeta = {'Strike':[],'Expiry':[]}
    for i in resultOptionBook:

        instrument_name = i['instrument_name']
        instrument_data = instrument_name.split("-")
        expiry = instrument_data[1]
        strike = float(instrument_data[2])
        if not strike in dictMeta['Strike']:
            dictMeta['Strike'].append(strike)
        if not expiry in dictMeta['Expiry']:
            dictMeta['Expiry'].append(expiry)
        if not strike in dictMeta:
            dictMeta[strike]=[]
        if not expiry in dictMeta:
            dictMeta[expiry]=[]
        if not strike in dictMeta[expiry]:
            dictMeta[expiry].append(strike)
        if not expiry in dictMeta[strike]:
            dictMeta[strike].append(expiry)
        dictMeta['Strike'].sort()
    return dictMeta

OptionMeta = GetOptionMeta()
OptionBook = GetOptionBook()

print(OptionMeta)

thisExpiry = "18NOV22"
thisStrikes = ["BTC-"+thisExpiry+"-"+str(int(i)) for i in sorted(OptionMeta[thisExpiry])]
for i in thisStrikes:
    thisPut = [item for item in OptionBook if item.get('instrument_name')==i+"-P"][0]
    thisCall = [item for item in OptionBook if item.get('instrument_name')==i+"-C"][0]
    print(i, thisPut['bid_IV'], thisCall['bid_IV'], thisPut['ask_IV'], thisCall['ask_IV'])
    
    
    
    

<ipython-input-7-3ad19a8fe729>:24: RuntimeWarning: divide by zero encountered in double_scalars
  thisIV = thisIV + (price - thisPx) / dPx * dIV


{'Strike': [5000.0, 8000.0, 10000.0, 11000.0, 12000.0, 13000.0, 14000.0, 15000.0, 16000.0, 16500.0, 17000.0, 17250.0, 17500.0, 17750.0, 18000.0, 18250.0, 18500.0, 18750.0, 19000.0, 19250.0, 19500.0, 19750.0, 20000.0, 20250.0, 20500.0, 21000.0, 21500.0, 22000.0, 22500.0, 23000.0, 23500.0, 24000.0, 25000.0, 26000.0, 27000.0, 28000.0, 29000.0, 30000.0, 31000.0, 32000.0, 33000.0, 34000.0, 35000.0, 36000.0, 38000.0, 40000.0, 42000.0, 45000.0, 50000.0, 55000.0, 60000.0, 65000.0, 70000.0, 80000.0, 90000.0, 100000.0, 120000.0, 140000.0, 150000.0, 200000.0, 250000.0, 300000.0], 'Expiry': ['18NOV22', '31MAR23', '25NOV22', '30DEC22', '29SEP23', '30JUN23', '10NOV22', '27JAN23', '11NOV22'], 21500.0: ['18NOV22', '10NOV22', '25NOV22', '11NOV22'], '18NOV22': [21500.0, 27000.0, 28000.0, 18500.0, 17500.0, 34000.0, 20500.0, 32000.0, 18000.0, 22000.0, 30000.0, 16000.0, 12000.0, 29000.0, 15000.0, 19500.0, 20000.0, 26000.0, 25000.0, 24000.0, 14000.0, 16500.0, 23000.0, 33000.0, 13000.0, 31000.0, 11000.0, 225

In [10]:
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

z_data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,0,27.809850,49.619360,83.080670,116.66320,130.41400,150.72060,220.18710,156.15360,148.64160,...,49.961420,21.89279,17.025520,11.743170,14.752260,13.667100,5.677561,3.312340,1.156517,-0.147662
1,1,27.719660,48.550220,65.213740,95.27666,116.99640,133.90560,152.34120,151.93400,160.11390,...,33.088710,38.40972,44.248430,69.578600,4.019351,3.050024,3.039719,2.996142,2.967954,1.999594
2,2,30.426700,33.477520,44.809530,62.47495,77.43523,104.21530,102.73930,137.00040,186.07060,...,48.471320,74.71461,60.090900,7.073525,6.089851,6.537450,6.666096,7.306965,5.736840,3.625628
3,3,16.665490,30.108600,39.969520,44.12225,59.57512,77.56929,106.89250,166.55390,175.23810,...,60.559160,55.92124,15.172840,8.248324,36.680870,61.934130,20.268670,68.588190,46.498120,0.236010
4,4,8.815617,18.351600,8.658275,27.58590,48.62691,60.18013,91.32860,145.71090,116.06530,...,47.426910,69.20731,44.954680,29.171970,17.916740,16.255150,14.655590,17.260480,31.222450,46.717040
5,5,6.628881,10.413390,24.819390,26.08952,30.16050,52.30802,64.71007,76.30823,84.63686,...,140.264700,81.26501,56.457560,30.421640,17.287820,8.302431,2.981626,2.698536,5.886086,5.268358
6,6,21.839750,6.639270,18.970850,32.89204,43.15014,62.86014,104.66570,130.22940,114.84940,...,122.422100,123.96980,109.095200,98.419560,77.613740,32.490310,14.673440,7.370775,0.037110,0.642339
7,7,53.343030,26.797970,6.639270,10.88787,17.20440,56.18116,79.70141,90.84530,98.27675,...,68.174900,46.24076,39.938570,31.216530,36.883350,40.025250,117.429700,12.703280,1.729771,0.000000
8,8,25.667850,63.057170,22.141400,17.07400,41.74483,60.27227,81.42432,114.44400,102.32340,...,59.193550,42.47175,14.635980,6.944074,6.944075,27.749360,0.000000,0.000000,0.094494,0.077323
9,9,12.827000,69.205540,46.762930,13.96517,33.88744,61.82613,84.74799,121.12200,145.27410,...,79.344250,25.93483,6.944074,6.944074,6.944075,7.553681,0.000000,0.000000,0.000000,0.000000
